In [4]:
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
from imp import reload
import sys, os
import scipy.special as sp
import dill
sys.path.append('../')
import mg_si
import csv   
import datetime


In [5]:
pl = mg_si.planet.Custom()

In [7]:
# times = np.linspace(0,1000e6*365.25*24*3600,1000)
times = np.linspace(0,4568e6*365.25*24*3600,4000)

## Parameters to tweak:
T_cmb0 = 5900 # 5000-6000
deltaT0 = 2800 # nominally 2800
## Initial Core State
X_Mg_0 = 0.02 # 0. - 0.05
X_Si_0 = 0.01 # 0. - 0.05
X_O_0 = 0.16  # 10. - 22.
## background mantle state
fraction_MgFe_b = 0.8
X_MgFeO_b = 0.05
X_SiO2_b = 0.01
## Mantle viscosity
nu_present = 10**19/pl.params.mantle.rho #[m^2/s]

T_cmbs = [5000, 5500, 6000]
X_Mgs = [1e-12, 0.01, 0.025, 0.05]
X_Sis = [1e-12, 0.01, 0.025, 0.05]
nus = np.array([10**19, 10**20, 10**21])/pl.params.mantle.rho
for nu_present in nus:  #[m^2/s]
    for X_Mg_0 in X_Mgs:
        for X_Si_0 in X_Sis:
            for T_cmb0 in T_cmbs:
                T_um0 = T_cmb0-deltaT0

                Moles_0 = pl.reactions.compute_Moles_0(X_Mg_0, X_Si_0, X_O_0, T_cmb0)
                x0 = [T_cmb0, T_um0]
                x0 = x0+Moles_0
                pl.params.reactions.Moles_0 = Moles_0

                Mm_b = pl.reactions.mantle.compute_Mm_b(fraction_MgFe_b, X_MgFeO_b, X_SiO2_b)
                pl.params.reactions.Mm_b = Mm_b

                T_present = 1350 # [K]
                nu_old =  nu_present/1e3
                T_old = T_um0
                A,nu0 = pl.mantle_layer.find_arrenhius_params(nu_present, T_present, nu_old, T_old, set_values=True)

                solution = pl.integrate(times, x0)

                filepath = '../computed_solutions/Tc{:d}_dT{:d}_XM{:.2f}_XS{:.2f}_XO{:.2f}_fMb{:.2f}_Xmb{:.2f}_XSb{:.2f}_nu{:.0e}/'.format(T_cmb0, deltaT0, 
                        X_Mg_0, X_Si_0, X_O_0, fraction_MgFe_b, X_MgFeO_b, X_SiO2_b, nu_present)
                if not os.path.exists('../computed_solutions/'):
                    os.mkdir('../computed_solutions/')
                if not os.path.exists(filepath):
                    os.mkdir(filepath)
                dill.dump((planet,time,solution), open(filepath+'data.m','wb'))

                mplt.temperature(pl, times, solution, filepath=filepath)
                mplt.coremoles(pl, times, solution, filepath=filepath)
                mplt.composition(pl, times, solution, filepath=filepath)

                time = str(datetime.datetime.now())
                r_i = pl.core_layer.r_i(solution[-1,0], one_off=True)
                csvdata = [time, T_cmb0, deltaT0, r_i, X_Mg_0, X_Si_0, X_O_0, fraction_MgFe_b, X_MgFeO_b, X_SiO2_b, nu_present]
                with open(r'../computed_solutions/run_data.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow(csvdata)

FileNotFoundError: [Errno 2] No such file or directory: '../computed_solutions/Tc5000_dT2800_XM0.00_XS0.00_XO0.16_fMb0.80_Xmb0.05_XSb0.01_nu2e+15/'